In [1]:
import numpy as np
import fermion as f
import matplotlib.pyplot as plt
from pandas import DataFrame
from fermion import Math as fm
from timeit import default_timer as timer

In [2]:
dq = f.Operator.double_quantum(50)
Z = f.Operator.global_Z(50)

In [3]:
time_evo = f.Unitary(dq,.05,2.05)

In [4]:
start = timer()
Z1 = time_evo.evolve_op(Z,0.05)
end = timer()
print(end - start)

0.010232099999999633


In [5]:
start = timer()
comm = Z1.commutator(Z).normal_order().compactify() ** 2
end = timer()
print(end - start)

103.3548038


In [6]:
start = timer()
tr1 = comm.trace()
end = timer()
print(end - start)

39.6503129


In [7]:
start = timer()
tr2 = (Z1.commutator(Z).normal_order().compactify() ** 2).trace()
end = timer()
print(end - start)

205.1364052


In [8]:
147.61/60

2.460166666666667

In [9]:
2/0.05

40.0

In [10]:
40*2.5/60

1.6666666666666667

In [11]:
_,T = dq.jordan_wigner()

In [12]:
start = timer()
fm.adj(T) @ Z.coef[2] @ T
end = timer()
print(end-start)

0.0004136000000016793


In [13]:
start = timer()
temp = fm.tensor_change_of_basis(Z.coef[2], T)
end = timer()
print(end-start)

0.003799899999989975


In [14]:
import tensorflow as tf

In [15]:
z_tf = tf.constant(Z.coef[2])

In [16]:
t_tf = tf.constant(T)

In [17]:
start = timer()
hold = tf.einsum("ij,ik,jl -> kl", z_tf,t_tf,t_tf)
end = timer()
print(end-start)

0.0070797999999854255


In [18]:
np.allclose(hold.numpy(),temp)

True

In [98]:
dq = f.Operator.double_quantum(10)
Z = f.Operator.global_Z(10)
time_evo = f.Unitary(dq,.05,2.05)
Z1 = time_evo.evolve_op(Z,0.05)

In [99]:
other = (Z1 * Z)
other

In [116]:
def tr_alt(op):
    tr = 0
    if 0 in op.components:
        tr += op.coef[0]
    if 2 in op.components:
        tr += 0.5 * (np.sum(np.diagonal(op.coef[2],op.n_fermion))+np.sum(np.diagonal(op.coef[2],-op.n_fermion)))
    if 4 in op.components:
        spr = tf.sparse.from_dense(op.coef[4])
        for idx, val in zip(spr.indices.numpy(), spr.values):
            if fm.fermion_weight(idx, op.n_fermion) == 0:
                tr += val * fm.trace_weight(idx, op.n_fermion)
    return tr

In [117]:
start = timer()
for i in range(10):
    other.trace()
end = timer()
print(end-start)

0.8771942000021227


In [118]:
start = timer()
for i in range(10):
    tr_alt(other)
end = timer()
print(end-start)

13.501828199980082


In [120]:
other.components

[4]

In [123]:
hold

<tf.Tensor: shape=(100, 100), dtype=float64, numpy=
array([[-1.45012041e-18, -8.49229289e-01,  1.57209315e-18, ...,
         1.30300501e-14,  1.32814766e-18,  1.38492326e-14],
       [ 8.49229289e-01,  2.71050543e-19, -5.08891820e-01, ...,
         7.04731412e-19,  3.93077498e-16,  5.96311195e-19],
       [ 1.57209315e-18,  5.08891820e-01,  1.73472348e-18, ...,
         8.96721933e-15, -1.08420217e-19,  9.11933289e-15],
       ...,
       [-1.30300501e-14,  7.04731412e-19, -8.96721933e-15, ...,
         4.33680869e-18,  5.08891820e-01, -2.71050543e-19],
       [ 1.32814766e-18, -3.93077498e-16, -1.08420217e-19, ...,
        -5.08891820e-01, -2.11419424e-18,  8.49229289e-01],
       [-1.38492326e-14,  5.96311195e-19, -9.11933289e-15, ...,
        -2.71050543e-19, -8.49229289e-01,  1.35525272e-18]])>

In [76]:
tr_alt(other)

(-77.14655599401775+0j)

In [79]:
spr = tf.sparse.from_dense(hold)

In [80]:
spr

In [82]:
spr.indices

<tf.Tensor: shape=(8928, 2), dtype=int64, numpy=
array([[ 0,  0],
       [ 0,  1],
       [ 0,  2],
       ...,
       [99, 97],
       [99, 98],
       [99, 99]], dtype=int64)>

In [86]:
test = tf.sparse.from_dense(fm.chop(hold))

In [112]:
test.indices.numpy()[2]

array([0, 5], dtype=int64)